# Deploying a project for offline inference
In this notebook we'll show how to create a deployment for a project that can be used to run inference locally, using OpenVINO

In [1]:
# As usual we'll connnect to the platform first

from sc_api_tools import SCRESTClient

client = SCRESTClient(
    host='https://10.91.242.203/',
    username='sc-api-tools@intel.com',
    password='Inteldemos!'
)

Authenticating on host https://10.91.242.203/...
Authentication successful. Cookie received.


### Selecting a project for deployment
Let's list all projects in the workspace and select one for which to create a deployment

In [2]:
from sc_api_tools.rest_managers import ProjectManager

project_manager = ProjectManager(session=client.session, workspace_id=client.workspace_id)
projects = project_manager.list_projects()

5 projects were found on the platform:

 Project: Bottle Anomaly Classification
  Task 1: Anomaly task
    Labels: ['Normal', 'Anomalous']


 Project: Bottle Quality Control
  Task 1: Classification task
    Labels: ['Good', 'Bad']


 Project: Weld classification
  Task 1: Classification task
    Labels: ['Good', 'bad', 'empty']


 Project: COCO animal detection demo
  Task 1: Detection task
    Labels: ['dog', 'horse', 'cat', 'No Object']


 Project: COCO anomalous animal demo
  Task 1: Detection task
    Labels: ['animal', 'No Object']
  Task 2: Anomaly classification task
    Labels: ['Normal', 'Anomalous']




## Deploying the project
Let's go with the project we created in notebook [004](004_create_pipeline_project_from_dataset.ipynb): `COCO anomalous animal demo`. To create a deployment, we can use the `client.deploy_project` convenience method. This will download the active (OpenVINO) models for all tasks in the project to our local machine, so that we can use them to run inference locally.

> **NOTE**: Downloading the model data may take some time, especially models for anomaly tasks are on the order of 100 Mb in size so please be prepared to wait a bit

In [3]:
PROJECT_NAME = 'COCO anomalous animal demo'

deployment = client.deploy_project(project_name=PROJECT_NAME)

Retrieving MO model data for Detection task...
Retrieving MO model data for Anomaly classification task...


### Preparing the models for inference
Now that the `deployment` is created and the models are saved to the local disk, we can load the models into memory to prepare them for inference. 

In [4]:
deployment.load_inference_models(device='CPU')

## Running inference on an image locally
Now, we can load an image as a numpy array (for instance using OpenCV) and use the `deployment.infer` method to generate a prediction for it

In [5]:
import cv2
import time

numpy_image = cv2.imread('data/dogs.png')

t_start = time.time()
prediction = deployment.infer(numpy_image)
t_elapsed = time.time() - t_start

print(f"Running local inference on image took {t_elapsed*1000:.2f} milliseconds")

Running local inference on image took 353.86 milliseconds


### Inspecting the result
The `Prediction` object generated by `deployment.infer` is equal in structure to the predictions sent by the platform. So let's have a closer look at it. We can do so in two ways: 

1. Visualise it using the `show_image_with_annotation_scene` utility function
2. Inspecting it's properties via the `prediction.overview` property

Let's show it on the image first

In [6]:
from sc_api_tools.utils import show_image_with_annotation_scene

show_image_with_annotation_scene(numpy_image, prediction)

In [7]:
print(prediction.overview)

{'annotations': [{'labels': [{'color': '#41405bff',
                              'name': 'animal',
                              'probability': 0.32638073},
                             {'color': '#ff5662ff',
                              'name': 'Anomalous',
                              'probability': 1.0}],
                  'shape': {'height': '0.310',
                            'type': 'RECTANGLE',
                            'width': '0.100',
                            'x': '0.722',
                            'y': '0.070'}},
                 {'labels': [{'color': '#41405bff',
                              'name': 'animal',
                              'probability': 0.20990069},
                             {'color': '#ff5662ff',
                              'name': 'Anomalous',
                              'probability': 1.0}],
                  'shape': {'height': '0.207',
                            'type': 'RECTANGLE',
                            'width': '0.062',
    

## Saving the deployment
When we create the deployment, the model data is saved to a temporary folder. We store the deployment for offline re-use later on by saving it: This will copy the model data from the temporary folder to the path we specify. If we want to run inference locally again, we can simply reload the deployment from the saved folder, without having to connect to the platform again.

In [12]:
import os

PATH_TO_DEPLOYMENT_FOLDER = os.path.join('deployments', PROJECT_NAME)

deployment.save(path_to_folder=PATH_TO_DEPLOYMENT_FOLDER)

## Loading a saved deployment
Loading a deployment that was previously saved to disk is easy and can be done without establishing a connection to the platform (or without even connecting to the internet, for that matter).

In [13]:
from sc_api_tools.deployment import Deployment

loaded_deployment = Deployment.from_folder(PATH_TO_DEPLOYMENT_FOLDER)

Again, to prepare the deployment for inference make sure to send the models to CPU (or whichever device you want to use)

In [14]:
loaded_deployment.load_inference_models(device='CPU')

That's it! Happy inferencing!